1. fs - Jalal
2. md - Shyma -- DONE https://www.dropbox.com/sh/gltzn82pth8zt9a/AAB28TClaUq6o7NnwIF0ijXsa?dl=0
3. fd - abdulelah --DONE https://www.dropbox.com/s/7d55xdq6q1u0g4k/fd.rar?dl=0
4. ms - Hussam

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
path = "./FIDs-features/"

def process(df, rType):
    df['FID'] = df['p1'].str[:5]
    df['p1'] = path+df['p1']
    df['p2'] = path+df['p2']
    df['p1'] = df['p1'].str.replace('.jpg', '.pkl')
    df['p2'] = df['p2'].str.replace('.jpg', '.pkl')
    
    sample1 = df.sample(n=df.shape[0], replace=True)[['p1','FID']].reset_index().drop('index',axis=1)
    sample2 = df.sample(n=df.shape[0], replace=True)[['p2','FID']].reset_index().drop('index',axis=1)
    sample2.columns = ['p2','FID2']
    unrelated = pd.concat([sample1,sample2],axis=1)
    unrelated = unrelated[unrelated['FID'] != unrelated['FID2']][['p1','p2']]
    
    ###############################################################
    # Related file
    p1 = df['p1'].values.tolist()
    p2 = df['p2'].values.tolist()
    F = pd.read_pickle(p1[0])
    C = pd.read_pickle(p2[0])
    related_fs_set = ((F + C) / np.power((F - C),2)).reshape(1,512)

    for r1, r2 in zip(p1[1:],p2[1:]):
        F = pd.read_pickle(r1).reshape(1,512)
        C = pd.read_pickle(r2).reshape(1,512)
        temp = (F + C) / np.power((F - C),2)
        related_fs_set = np.append(related_fs_set, temp, axis=0)
        
    #related_fs_set / np.sqrt(np.sum(related_fs_set, axis=0))
    pd.DataFrame(related_fs_set).to_pickle("Related_{}.pkl".format(rType))
        
    ###############################################################
    # UnRelated file
    p1 = unrelated['p1'].values.tolist()
    p2 = unrelated['p2'].values.tolist()
    F = pd.read_pickle(p1[0])
    C = pd.read_pickle(p2[0])
    unrelated_fs_set = ((F + C) / np.power((F - C),2)).reshape(1,512)

    for r1, r2 in zip(p1[1:],p2[1:]):
        F = pd.read_pickle(r1).reshape(1,512)
        C = pd.read_pickle(r2).reshape(1,512)
        temp = (F + C) / np.power((F - C),2)
        unrelated_fs_set = np.append(unrelated_fs_set, temp, axis=0)
        
    #unrelated_fs_set / np.sqrt(np.sum(unrelated_fs_set, axis=0))
    pd.DataFrame(unrelated_fs_set).to_pickle("Unrelated_{}.pkl".format(rType))
  
# Example
df = pd.read_pickle("./lists/pairs/pickles/Direct/fs-faces.pkl")
process(df, "fs")